### Problem 54: Poker Hands
<p>In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:</p>
<ul><li><b>High Card</b>: Highest value card.</li>
<li><b>One Pair</b>: Two cards of the same value.</li>
<li><b>Two Pairs</b>: Two different pairs.</li>
<li><b>Three of a Kind</b>: Three cards of the same value.</li>
<li><b>Straight</b>: All cards are consecutive values.</li>
<li><b>Flush</b>: All cards of the same suit.</li>
<li><b>Full House</b>: Three of a kind and a pair.</li>
<li><b>Four of a Kind</b>: Four cards of the same value.</li>
<li><b>Straight Flush</b>: All cards are consecutive values of same suit.</li>
<li><b>Royal Flush</b>: Ten, Jack, Queen, King, Ace, in same suit.</li>
</ul><p>The cards are valued in the order:<br>2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.</p>
<p>If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.</p>
<p>Consider the following five hands dealt to two players:</p>
<div class="center">
<table><tr><td><b>Hand</b></td><td> </td><td><b>Player 1</b></td><td> </td><td><b>Player 2</b></td><td> </td><td><b>Winner</b></td>
</tr><tr><td><b>1</b></td><td> </td><td>5H 5C 6S 7S KD<br><div class="smaller">Pair of Fives</div></td><td> </td><td>2C 3S 8S 8D TD<br><div class="smaller">Pair of Eights</div></td><td> </td><td>Player 2</td>
</tr><tr><td><b>2</b></td><td> </td><td>5D 8C 9S JS AC<br><div class="smaller">Highest card Ace</div></td><td> </td><td>2C 5C 7D 8S QH<br><div class="smaller">Highest card Queen</div></td><td> </td><td>Player 1</td>
</tr><tr><td><b>3</b></td><td> </td><td>2D 9C AS AH AC<br><div class="smaller">Three Aces</div></td><td> </td><td>3D 6D 7D TD QD<br><div class="smaller">Flush  with Diamonds</div></td><td> </td><td>Player 2</td>
</tr><tr><td><b>4</b></td><td> </td><td>4D 6S 9H QH QC<br><div class="smaller">Pair of Queens<br>Highest card Nine</div></td><td> </td><td>3D 6D 7H QD QS<br><div class="smaller">Pair of Queens<br>Highest card Seven</div></td><td> </td><td>Player 1</td>
</tr><tr><td><b>5</b></td><td> </td><td>2H 2D 4C 4D 4S<br><div class="smaller">Full House<br>With Three Fours</div></td><td> </td><td>3C 3D 3S 9S 9D<br><div class="smaller">Full House<br>with Three Threes</div></td><td> </td><td>Player 1</td>
</tr></table></div>
<p>The file, <a href="resources/documents/0054_poker.txt">poker.txt</a>, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.</p>
<p>How many hands does Player 1 win?</p>

In [105]:
import os

In [118]:
with open(os.path.join(os.path.dirname(os.getcwd()),'xxx_ProjectEulerData','0054_poker.txt'), 'r') as file:
    file_content = [x.split(' ') for x in file.read().split('\n') if x != '']
player_hands = [(x[0:5],x[5:]) for x in file_content]

In [249]:
def card_value(card):
    """Assign numeric values to cards."""
    values = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'T': 8, 'J': 9, 'Q': 10, 'K': 11, 'A': 12}
    return values[card[0]]

def is_straight(cards):
    """Check if the hand is a straight."""
    values = sorted([card_value(card) for card in cards])
    # Check for the Ace as both high and low
    if values == [0, 1, 2, 3, 12]:
        return True
    return values == list(range(min(values), max(values) + 1))

def rank_hand(hand):
    """Assign a numeric rank to a poker hand."""
    cards = sorted(hand, key=card_value, reverse=True)

    # Create a dictionary to count the number of each card rank
    card_counts = {}
    for card in cards:
        rank = card[0]
        card_counts[rank] = card_counts.get(rank, 0) + 1

    # Check for flush and straight flush
    if is_flush(cards) and is_straight(cards):
        if cards[0][0] == 'A':
            return 'C' + '0'  # Royal flush
        else:
            straight_rank = hex(card_value(cards[0]))[2:]
            return 'B' + straight_rank.zfill(2)  # Straight flush

    # Check for four of a kind
    for rank, count in card_counts.items():
        if count == 4:
            four_of_a_kind_rank = hex(card_value(rank + 'H'))[2:]
            kicker_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card, counts in card_counts.items() if counts==1])
            return 'A' + four_of_a_kind_rank.zfill(2)  + kicker_rank # Four of a kind

    # Check for full house
    three_of_a_kind = None
    pair = None
    for rank, count in card_counts.items():
        if count == 3:
            three_of_a_kind = rank
        elif count == 2:
            pair = rank
    if three_of_a_kind is not None and pair is not None:
        return '9' + hex(card_value(three_of_a_kind))[2:].zfill(2) + hex(card_value(pair))[2:].zfill(3)  # Full house

    # Check for flush
    if is_flush(cards):
        high_card_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card in cards])
        return '8' + high_card_rank  # Flush

    # Check for straight
    if is_straight(cards):
        if card_value(cards[0]) == 12:
            return '7' + '00'  # Straight with Ace as low
        straight_rank = hex(card_value(cards[0]))[2:]
        return '7' + straight_rank.zfill(5)  # Straight

    # Check for three of a kind
    for rank, count in card_counts.items():
        if count == 3:
            three_of_a_kind_rank = hex(card_value(rank + 'H'))[2:]
            kicker_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card, counts in card_counts.items() if counts==1])
            return '6' + three_of_a_kind_rank.zfill(2) + kicker_rank  # Three of a kind

    # Check for two pairs
    pairs = []
    for rank, count in card_counts.items():
        if count == 2:
            pairs.append(rank)
    if len(pairs) == 2:
        kicker_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card, counts in card_counts.items() if counts==1])
        return '5' + hex(card_value(max(pairs) + 'H'))[2:].zfill(2) + hex(card_value(min(pairs) + 'H'))[2:].zfill(2) + kicker_rank  # Two pair

    # Check for one pair
    for rank, count in card_counts.items():
        if count == 2:
            kicker_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card, counts in card_counts.items() if counts==1])
            pair_rank = hex(card_value(rank + 'H'))[2:]
            return '4' + pair_rank.zfill(2) + kicker_rank  # One pair

    # High card
    high_card_rank = ''.join([hex(card_value(card[0] + 'H'))[2:] for card in cards])
    return '1' + high_card_rank.zfill(6)


def hand_format(hand):
    rank = '0x'+rank_hand(hand).ljust(6,'0')
    return int(rank,16)

In [250]:
def pe_54():
    result_dict = {'Player 1 Win':0,'Player 2 Win':0,'Tie':0}
    for hand in player_hands:
        rank1 = rank_hand(hand[0])
        rank2 = rank_hand(hand[1])

        if rank2 < rank1:
            result_dict['Player 1 Win'] += 1
        elif rank2 > rank1:
            result_dict['Player 2 Win'] += 1
        else:
            result_dict['Tie'] += 1
            print(hand[0], rank1, hand[1],rank2)
    return result_dict

In [254]:
print(pe_54())

{'Player 1 Win': 376, 'Player 2 Win': 624, 'Tie': 0}
